# Bitcoin price forecast

### ATENTION: This notebook has an error applying the scaler. The scaler should be fitted to the training data and then applied to all the data. This is not done in this notebook.

In [ ]:
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
# dataset @ https://finance.yahoo.com/quote/BTC-USD/history/
df = pd.read_csv('BTC-USD.csv')
df.head()

In [ ]:
df = df[['Date', 'Close']]
df.head()

In [ ]:
def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
datetime_object

In [ ]:
df['Date'] = df['Date'].apply(str_to_datetime)
df['Date'].head()

In [ ]:
df.index = df.pop('Date')
df.head()

In [ ]:
period_in_days = 10
df['Change'] = (df['Close'].shift(-period_in_days) - df['Close']) / df['Close']
df.drop(df.index[-period_in_days:], inplace=True)
df

In [ ]:
change_percentage = 4
df['Change_bin'] = (df['Change'] >= change_percentage/100).astype(int)
df.head()

In [ ]:
df['Change_bin'].hist()
plt.show()

In [ ]:
plt.plot(df.index, df['Change'])

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
change_scaled = scaler.fit_transform(df['Change'].values.reshape(-1,1))
close_scaled = scaler.fit_transform(df['Close'].values.reshape(-1,1))
df['Change'] = change_scaled
df['Close'] = close_scaled
df.head()

In [ ]:
import numpy as np

# n = number of features
def df_to_windowed_df(dataframe, n=3):
  column_features = dataframe.columns[:-1]
  target = dataframe.loc[:,df.columns[-1]].to_numpy()
  dates = dataframe.index[n-1:].to_numpy()
  Y = target[n-1:]
  feat_amount = len(df.columns) - 1
  input_columns = []
  for i in range(1, feat_amount + 1):
    input_columns.append(dataframe.loc[:,dataframe.columns[-1 * i - 1]].to_numpy())

  X = []
  # X[j] correspond to the j-th feature
  for i in range(feat_amount):
    X.append([])
  for i in range(n):
    for j in range(feat_amount):
      if i == n - 1:
        X[j].append(input_columns[j][i:])
      else:
        X[j].append(input_columns[j][i:i - n + 1])

  ret_df = pd.DataFrame()
  ret_df['Date'] = dates

  for i in range(0, n):
    for j in range(feat_amount):
      ret_df[f'Target-{column_features[j]}-{n-i}'] = X[j][i]
      
  ret_df['Target'] = Y

  return ret_df

windowed_df = df_to_windowed_df(df, n=10)
windowed_df.head()

In [ ]:
def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]

  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, y = windowed_df_to_date_X_y(windowed_df)
dates.shape, X.shape, y.shape

In [ ]:
train_percentage = 0.7
val_percentage = 0.1
test_percentage = 0.2

assert train_percentage + val_percentage + test_percentage == 1

lim_train = int(len(dates) * train_percentage)
lim_val = int(len(dates) * val_percentage)

dates_train, X_train, y_train = dates[:lim_train], X[:lim_train], y[:lim_train]
dates_val, X_val, y_val = dates[lim_train: lim_train + lim_val], X[lim_train: lim_train + lim_val], y[lim_train: lim_train + lim_val]
dates_test, X_test, y_test = dates[ lim_train + lim_val:], X[ lim_train + lim_val:], y[ lim_train + lim_val:]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
from tensorflow.keras import layers, regularizers
from sklearn.utils.class_weight import compute_class_weight

# Suponiendo que y_train es tu conjunto de etiquetas de entrenamiento
# Calcula los pesos de las clases
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(class_weight_dict)

# Modificaciones en la arquitectura del modelo
model = Sequential()
model.add(layers.LSTM(32, input_shape=(20, 1), activation='tanh'))  # Uso de tanh y sigmoid
model.add(layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

# Ajustes en la tasa de aprendizaje y el decaimiento
lr_schedule = InverseTimeDecay(0.005,
                               decay_steps=1000,
                               decay_rate=0.95,
                               staircase=True)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule),
              metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, class_weight=class_weight_dict)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_train)

# Threshold para convertir la salida del modelo en una predicción binaria
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

# Calcular la matriz de confusión
cm = confusion_matrix(y_train, y_pred_binary)

# Imprimir la matriz de confusión
print(cm)

# Graficar la matriz de confusión
plt.imshow(cm, cmap='Blues')
plt.colorbar()
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_train)
y_pred = np.round(y_pred)

# confusion matrix
import sklearn.metrics as skm

cm = confusion_matrix(y_train, y_pred)
cr = classification_report(y_train, y_pred)

print("Matriz de Confusión:")
print(cm)
print("\nClassification Report:")
print(cr)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)

# confusion matrix
import sklearn.metrics as skm

cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print("Matriz de Confusión:")
print(cm)
print("\nClassification Report:")
print(cr)